# easylora Quickstart

This notebook demonstrates a minimal LoRA fine-tuning run using easylora.
It uses a tiny model and a small synthetic dataset so it runs on CPU in under a minute.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alexsuw/easylora/blob/main/notebooks/quickstart_colab.ipynb)

In [ ]:
# Install easylora (uncomment in Colab)
# !pip install easylora

In [ ]:
import json
from pathlib import Path

import easylora

print(f"easylora version: {easylora.__version__}")

## 1. Create a tiny training dataset

In [ ]:
data_path = Path("tiny_train.jsonl")
examples = [
    {"text": "The quick brown fox jumps over the lazy dog."},
    {"text": "A simple test sentence for training."},
    {"text": "LoRA adapters are parameter efficient."},
    {"text": "Fine-tuning language models is fun."},
]
data_path.write_text("\n".join(json.dumps(ex) for ex in examples))
print(f"Created {data_path} with {len(examples)} examples")

## 2. Configure and train

In [ ]:
from easylora import TrainConfig, train
from easylora.config import (
    DataConfig,
    ModelConfig,
    OutputConfig,
    TrainLoopConfig,
)

config = TrainConfig(
    model=ModelConfig(
        base_model="hf-internal-testing/tiny-random-LlamaForCausalLM",
        device_map=None,
        torch_dtype="fp32",
    ),
    data=DataConfig(
        dataset_path=str(data_path),
        format="raw",
        max_seq_len=64,
    ),
    training=TrainLoopConfig(
        epochs=1,
        batch_size=2,
        grad_accum=1,
        max_steps=3,
        logging_steps=1,
        gradient_checkpointing=False,
    ),
    output=OutputConfig(
        output_dir="./colab_output",
        allow_overwrite=True,
    ),
)

artifacts = train(config)
print(f"\nAdapter saved to: {artifacts.adapter_dir}")

## 3. Inspect artifacts

In [ ]:
summary = json.loads(Path(artifacts.summary_path).read_text())
print("Training summary:")
for k, v in summary.items():
    print(f"  {k}: {v}")

In [ ]:
print("\nAdapter files:")
for f in sorted(Path(artifacts.adapter_dir).iterdir()):
    size = f.stat().st_size
    print(f"  {f.name} ({size:,} bytes)")

## 4. Generate text with the adapter

In [ ]:
from easylora import load_adapter
from easylora.config import ModelConfig
from easylora.eval.generate import generate_samples
from easylora.utils.hf import load_tokenizer

model = load_adapter(
    "hf-internal-testing/tiny-random-LlamaForCausalLM",
    artifacts.adapter_dir,
    device_map=None,
)
tokenizer = load_tokenizer(
    ModelConfig(base_model="hf-internal-testing/tiny-random-LlamaForCausalLM")
)

outputs = generate_samples(
    model,
    tokenizer,
    ["The quick brown"],
    max_new_tokens=20,
    do_sample=False,
)
print(f"Generated: {outputs[0]}")

In [ ]:
# Clean up
import shutil

shutil.rmtree("./colab_output", ignore_errors=True)
data_path.unlink(missing_ok=True)
print("Done!")